In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import chess.pgn as pgn
import chess
import numpy as np
from math import ceil
from sklearn.feature_extraction import DictVectorizer
from collections import defaultdict, Counter

In [3]:
data=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_games.csv')
data.Scores=data.Scores.str.replace(' ','').str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Scores=data.Scores.apply(lambda x: list(map(int,x.split())))
data.Scores=data.Scores.apply(lambda x: list(map(int,x)))
data.Moves=data.Moves.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Moves=data.Moves.apply(lambda x: x.split())
data.UCI=data.UCI.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.UCI=data.UCI.apply(lambda x: x.split())
data.head()

,Result,Moves,UCI,Scores,NumMoves,MeanScore,ModeScore,FinalScore
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[g1f3, g8f6, c2c4, c7c5, b2b3, g7g6, c1b2, f8g...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...",19,23.947368,3,54
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[e2e4, d7d5, e4d5, g8f6, d2d4, f6d5, g1f3, g7g...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...",53,-692.198113,18,-11544
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[c2c4, g8f6, b1c3, d7d6, d2d4, e7e5, g1f3, b8d...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...",39,49.246753,31,196
3,1-0,"[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...","[e2e4, c7c5, g1f3, d7d6, b2b4, g8f6, b4c5, f6e...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...",25,306.040816,18,875
4,1/2-1/2,"[Nf3, d5, d4, Nf6, c4, e6, Nc3, Be7, Bg5, h6, ...","[g1f3, d7d5, d2d4, g8f6, c2c4, e7e6, b1c3, f8e...","[18, 29, 10, 22, 20, 18, 19, 28, 24, 18, 17, 1...",29,30.827586,18,-11


In [4]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,57,4765
1,1915,1999,1957,84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [5]:
features=data.copy()

'''
# Может попробовать так? (долго думает, пока непонятно надо это или нет)
wins = pd.DataFrame(columns = ["WhiteWins", "BlackWins"])
features=features.append(wins)
features.update(features[["WhiteWins", "BlackWins"]].fillna(0, inplace=False), overwrite=False)

for i,x in enumerate(features['Result']):
    if (x == "1/2-1/2"):
        features['WhiteWins'][i] = 1
        features['BlackWins'][i] = 1
    elif (x == "1-0"):
        features['WhiteWins'][i] = 1
    else:
        features['BlackWins'][i] = 1
        '''

features.Result=features.Result.apply(lambda x: 0.5 if x=='1/2-1/2' else x)
features.Result=features.Result.apply(lambda x: 1 if x=='1-0' else x)
features.Result=features.Result.apply(lambda x: 0 if x=='0-1' else x)

features['FullMoves']=features.Moves.apply(lambda x: len(x))

features=features.drop(columns=['Moves','Scores','UCI'])
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves
0,0.5,19,23.947368,3,54,38
1,0.0,53,-692.198113,18,-11544,106
2,1.0,39,49.246753,31,196,77
3,1.0,25,306.040816,18,875,49
4,0.5,29,30.827586,18,-11,58


In [6]:
data['WhiteMoves']=data.Moves.apply(lambda x: x[::2])
data['WhiteMovesUCI']=data.UCI.apply(lambda x: x[::2])
data['WhiteScores']=data.Scores.apply(lambda x: x[::2])
data['BlackMoves']=data.Moves.apply(lambda x: x[1::2])
data['BlackMovesUCI']=data.UCI.apply(lambda x: x[1::2])
data['BlackScores']=data.Scores.apply(lambda x: x[1::2])
data['DiffScores']=data.Scores.apply(lambda x: np.diff(x).tolist())
data['DiffWhiteScores']=data.DiffScores.apply(lambda x: x[::2])
data['DiffBlackScores']=data.DiffScores.apply(lambda x: x[1::2])

data=data.drop(columns=['NumMoves','MeanScore','ModeScore','FinalScore'])
data.head(3)

,Result,Moves,UCI,Scores,WhiteMoves,WhiteMovesUCI,WhiteScores,BlackMoves,BlackMovesUCI,BlackScores,DiffScores,DiffWhiteScores,DiffBlackScores
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[g1f3, g8f6, c2c4, c7c5, b2b3, g7g6, c1b2, f8g...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...","[Nf3, c4, b3, Bb2, e3, Be2, O-O, Nc3, Qc2, Rac...","[g1f3, c2c4, b2b3, c1b2, e2e3, f1e2, e1g1, b1c...","[18, 12, -5, 3, 22, 20, 8, 11, -6, 1, -21, -26...","[Nf6, c5, g6, Bg7, O-O, b6, Bb7, Nc6, Rc8, d5,...","[g8f6, c7c5, g7g6, f8g7, e8g8, b7b6, c8b7, b8c...","[17, 8, 12, -2, 21, 13, 21, 3, 5, -10, -1, 18,...","[-1, -5, -4, -13, 17, -9, -5, 24, -1, -1, -7, ...","[-1, -4, 17, -5, -1, -7, 13, -8, 11, -11, 20, ...","[-5, -13, -9, 24, -1, -5, -10, -9, -4, -11, -2..."
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[e2e4, d7d5, e4d5, g8f6, d2d4, f6d5, g1f3, g7g...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...","[e4, exd5, d4, Nf3, Be2, c4, Nc3, O-O, Be3, d5...","[e2e4, e4d5, d2d4, g1f3, f1e2, c2c4, b1c3, e1g...","[26, 68, 65, 48, 61, 63, 53, 69, 30, -2, -11, ...","[d5, Nf6, Nxd5, g6, Bg7, Nb6, O-O, Nc6, Bg4, B...","[d7d5, g8f6, f6d5, g7g6, f8g7, d5b6, e8g8, b8c...","[51, 57, 77, 93, 63, 58, 46, 29, 27, 12, 0, 24...","[25, 17, -11, 8, 12, -29, 45, -32, 2, 0, -5, -...","[25, -11, 12, 45, 2, -5, -7, -40, -3, 14, 11, ...","[17, 8, -29, -32, 0, -5, 23, 1, -29, -23, -17,..."
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[c2c4, g8f6, b1c3, d7d6, d2d4, e7e5, g1f3, b8d...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...","[c4, Nc3, d4, Nf3, Bg5, e3, Qc2, Bd3, h4, dxe5...","[c2c4, b1c3, d2d4, g1f3, c1g5, e2e3, d1c2, f1d...","[2, 5, 35, 37, 10, 8, 30, 13, -12, 13, -22, -1...","[Nf6, d6, e5, Nbd7, Be7, c6, O-O, h6, b5, dxe5...","[g8f6, d7d6, e7e5, b8d7, f8e7, c7c6, e8g8, h7h...","[21, 53, 45, 54, 22, 48, 17, 35, 31, 31, 20, -...","[19, -16, 48, -18, 10, -8, 17, -44, 12, -14, 4...","[19, 48, 10, 17, 12, 40, -13, 22, 43, 18, 42, ...","[-16, -18, -8, -44, -14, -18, -4, -47, -18, -5..."


#### Условные разделения на дебют, миттельшпиль и эндшпиль.
##### (1-15 - дебют, 16-35 - миттельшпиль, 36+ эндшпиль)
Разделения очень-очень условные, но имеют основания, так как игроки могут по-разному показывать себя в разных частях партии.

In [7]:
data = data.append(pd.DataFrame(columns = ["DebScores", "MitScores", "EndScores"]))

data['DebScores']=data.Scores.apply(lambda x: x[:16])
data['MitScores']=data.Scores.apply(lambda x: x[16:36])
data['EndScores']=data.Scores.apply(lambda x: x[36:])


modes=['DebScores','MitScores','EndScores']
var=['White','Black','Diff','DiffWhite','DiffBlack']

for m in modes:
    data['White' + m]=data[m].apply(lambda x: x[::2])
    data['Black' + m]=data[m].apply(lambda x: x[1::2])
    data['Diff'+m]=data[m].apply(lambda x: np.diff(x).tolist())
    data['DiffWhite'+m]=data['Diff'+m].apply(lambda x: x[::2])
    data['DiffBlack'+m]=data['Diff'+m].apply(lambda x: x[1::2])


In [8]:
data.head()

,BlackMoves,BlackMovesUCI,BlackScores,DebScores,DiffBlackScores,DiffScores,DiffWhiteScores,EndScores,MitScores,Moves,...,WhiteMitScores,BlackMitScores,DiffMitScores,DiffWhiteMitScores,DiffBlackMitScores,WhiteEndScores,BlackEndScores,DiffEndScores,DiffWhiteEndScores,DiffBlackEndScores
0,"[Nf6, c5, g6, Bg7, O-O, b6, Bb7, Nc6, Rc8, d5,...","[g8f6, c7c5, g7g6, f8g7, e8g8, b7b6, c8b7, b8c...","[17, 8, 12, -2, 21, 13, 21, 3, 5, -10, -1, 18,...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...","[-5, -13, -9, 24, -1, -5, -10, -9, -4, -11, -2...","[-1, -5, -4, -13, 17, -9, -5, 24, -1, -1, -7, ...","[-1, -4, 17, -5, -1, -7, 13, -8, 11, -11, 20, ...","[35, 54]","[-6, 5, 1, -10, -21, -1, -26, 18, 48, 48, 53, ...","[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...",...,"[-6, 1, -21, -26, 48, 53, 46, 51, 54, 70]","[5, -10, -1, 18, 48, 73, 68, 60, 46, 62]","[11, -4, -11, -11, 20, -25, 44, 30, 0, 5, 20, ...","[11, -11, 20, 44, 0, 20, 22, 9, -8, -8]","[-4, -11, -25, 30, 5, -27, -17, -6, 24]",[35],[54],[19],[19],[]
1,"[d5, Nf6, Nxd5, g6, Bg7, Nb6, O-O, Nc6, Bg4, B...","[d7d5, g8f6, f6d5, g7g6, f8g7, d5b6, e8g8, b8c...","[51, 57, 77, 93, 63, 58, 46, 29, 27, 12, 0, 24...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...","[17, 8, -29, -32, 0, -5, 23, 1, -29, -23, -17,...","[25, 17, -11, 8, 12, -29, 45, -32, 2, 0, -5, -...","[25, -11, 12, 45, 2, -5, -7, -40, -3, 14, 11, ...","[18, 20, 10, 2, 7, 22, -28, -26, -30, -34, -29...","[30, 27, -2, 12, -11, 0, -17, 24, 5, 15, 16, 3...","[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...",...,"[30, -2, -11, -17, 5, 16, 44, 18, 27, 18]","[27, 12, 0, 24, 15, 31, 25, 20, 26, 14]","[-3, -29, 14, -23, 11, -17, 41, -19, 10, 1, 15...","[-3, 14, 11, 41, 10, 15, -19, 2, -1, -4]","[-29, -23, -17, -19, 1, 13, -7, 7, -8]","[18, 10, 7, -28, -30, -29, -49, -51, -82, -83,...","[20, 2, 22, -26, -34, -28, -55, -50, -74, -77,...","[2, -10, -8, 5, 15, -50, 2, -4, -4, 5, 1, -21,...","[2, -8, 15, 2, -4, 1, -6, 1, 8, 6, 20, 6, 13, ...","[-10, 5, -50, -4, 5, -21, 4, -32, -9, -15, -33..."
2,"[Nf6, d6, e5, Nbd7, Be7, c6, O-O, h6, b5, dxe5...","[g8f6, d7d6, e7e5, b8d7, f8e7, c7c6, e8g8, h7h...","[21, 53, 45, 54, 22, 48, 17, 35, 31, 31, 20, -...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...","[-16, -18, -8, -44, -14, -18, -4, -47, -18, -5...","[19, -16, 48, -18, 10, -8, 17, -44, 12, -14, 4...","[19, 48, 10, 17, 12, 40, -13, 22, 43, 18, 42, ...","[-10, 12, -11, 0, -11, 29, 27, 31, 26, 24, 28,...","[-12, 31, 13, 31, -22, 20, -18, -30, -27, 6, -...","[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...",...,"[-12, 13, -22, -18, -27, -11, 53, 56, 68, 44]","[31, 31, 20, -30, 6, 74, 50, 55, 58, 59]","[43, -18, 18, -53, 42, -38, -12, 3, 33, -17, 8...","[43, 18, 42, -12, 33, 85, -3, -1, -10, 15]","[-18, -53, -38, 3, -17, -21, 6, 13, -14]","[-10, -11, -11, 27, 26, 28, 36, 46, 44, 40, 34...","[12, 0, 29, 31, 24, 42, 38, 36, 45, 36, 68, 53...","[22, -23, 11, -11, 40, -2, 4, -5, -2, 4, 14, -...","[22, 11, 40, 4, -2, 14, 2, -10, 1, -4, 34, -2,...","[-23, -11, -2, -5, 4, -6, 8, 8, -5, -2, -13, -..."
3,"[c5, d6, Nf6, Nxe4, Qb6, Bg4, Nxd6, Nc6, Nb4, ...","[c7c5, d7d6, g8f6, f6e4, d8b6, c8g4, e4d6, b8c...","[64, 53, 20, 20, 49, 95, 82, 93, 59, 142, 193,...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...","[-29, -35, -2, -10, 11, -4, 1, -35, 39, 4, -72...","[38, -29, 18, -35, 2, -2, 2, -10, 39, 11, 35, ...","[38, 18, 2, 2, 39, 35, -9, 10, 1, 44, 47, 8, 1...","[505, 567, 525, 834, 819, 870, 848, 860, 826, ...","[58, 59, 98, 142, 146, 193, 121, 129, 135, 136...","[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...",...,"[58, 98, 146, 121, 135, 140, 158, 186, 227, 500]","[59, 142, 193, 129, 136, 152, 210, 235, 521, 489]","[1, 39, 44, 4, 47, -72, 8, 6, 1, 4, 12, 6, 52,...","[1, 44, 47, 8, 1, 12, 52, 49, 294, -11]","[39, 4, -72, 6, 4, 6, -24, -8, -21]","[505, 525, 819, 848, 826, 830, 875]","[567, 834, 870, 860, 917, 868]","[62, -42, 309, -15, 51, -22, 12, -34, 91, -87,...","[62, 309, 51, 12, 91, 38]","[-42, -15, -22, -34, -87, 7]"
4,"[d5, Nf6, e6,

In [9]:
stats = [np.min, np.max, np.mean, lambda x: np.median(np.abs(x)),np.std, 
         lambda x: round(np.var(x),2), lambda x: max(set(x), key=x.count)]
stat_names = ['Min', 'Max', 'Mean', 'Median','Std','Variance','Mode']
color=['White','Black']

# общие оценки
for stat, stat_name in zip(stats, stat_names):
    features[stat_name + 'Score'] = data.Scores.apply(lambda x: stat(x))
    
# общие оценки по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + 'Score'] = data[c+'Scores'].apply(lambda x: stat(x))
        
    features['Final' +c+ 'Score']=data[c+'Scores'].apply(lambda x: x[-1])


# по разнцие в оценках в общем
for stat, stat_name in zip(stats, stat_names):
    features[stat_name + 'DeltaScore'] = data.DiffScores.apply(lambda x: stat(x))
    
# по разнцие в оценках по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + 'DeltaScore'] = data['Diff'+c+'Scores'].apply(lambda x: stat(x))
    
##########################################################################################################################    
# по частям партий (и все тоже самое, что выше)
modes=['Deb','Mit','End']

for m in modes:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + m+'Score'] = data[m+'Scores'].apply(lambda x: stat(x) if  len(x)!=0 else -1)

# по цветам
for c in color:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + c + m+'Score'] = data[c+m+'Scores'].apply(lambda x: stat(x) if  len(x)!=0 else -1)

    features['Final' +c+ m+'Score']=data[c+m+'Scores'].apply(lambda x: x[-1]  if  len(x)!=0 else 0)

# по разнцие в оценках
for m in modes:
    for stat, stat_name in zip(stats, stat_names):
        features[stat_name + m+'DeltaScore'] = data['Diff'+m+'Scores'].apply(lambda x: stat(x)  if  len(x)!=0 else -1)
        
# по разнцие в оценках по цветам
for c in color:
    for m in modes:
        for stat, stat_name in zip(stats, stat_names):
            features[stat_name + c + m+'DeltaScore'] = data['Diff'+c+m+'Scores'].apply(lambda x: stat(x)  if  len(x)!=0 else -1)    


In [10]:
print(features.shape)
features.head(3)

(24255, 148)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,StdBlackMitDeltaScore,VarianceBlackMitDeltaScore,ModeBlackMitDeltaScore,MinBlackEndDeltaScore,MaxBlackEndDeltaScore,MeanBlackEndDeltaScore,MedianBlackEndDeltaScore,StdBlackEndDeltaScore,VarianceBlackEndDeltaScore,ModeBlackEndDeltaScore
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,18.886274,356.69,5,-1,-1,-1.000000,-1.0,-1.000000,-1.00,-1
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,13.353688,178.32,1,-9578,92,-428.235294,19.5,1643.924055,2702486.30,5
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,19.917112,396.69,3,-31,31,-1.450000,8.0,14.623526,213.85,8


#### Индекс первого и последнего хода, когда белые или черные получили преимущество (>100 cp)
+победили ли они после получения преимущества (если не было преимущества - то сверяем с ничьей)

In [11]:
def find_advantage(scores):
    for i,x in enumerate(scores):
        if abs(x) >=100:
            return i
    return 0

def is_realised_advantage(ind,result,num=0):
    if ind == num and num != 0:
        return 1
    elif ind==0 and result==0.5:
        return 1
    elif ind!=0 and ind %2!=0 and result==1:
        return 1
    elif ind!=0 and ind %2==0 and result==0:
        return 1
    else:
        return 0        
    
    
features['FirstAdvantageInd']=data.Scores.apply(find_advantage)
features['IsFirstAdvantageRealised']=features[['FirstAdvantageInd','Result']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['FirstAdvantageInd']=features['FirstAdvantageInd'].apply(lambda x: ceil(x/2))

features['LastAdvantageInd']=data.Scores.apply(lambda x: np.argmax(np.absolute(x) >=100) or len(x))
features['IsLastAdvantageRealised']=features[['LastAdvantageInd','Result','NumMoves']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['LastAdvantageInd']=features['LastAdvantageInd'].apply(lambda x: ceil(x/2))

features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,MaxBlackEndDeltaScore,MeanBlackEndDeltaScore,MedianBlackEndDeltaScore,StdBlackEndDeltaScore,VarianceBlackEndDeltaScore,ModeBlackEndDeltaScore,FirstAdvantageInd,IsFirstAdvantageRealised,LastAdvantageInd,IsLastAdvantageRealised
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,-1,-1.000000,-1.0,-1.000000,-1.00,-1,0,1,19,0
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,92,-428.235294,19.5,1643.924055,2702486.30,5,29,1,29,1
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,31,-1.450000,8.0,14.623526,213.85,8,33,1,33,1
3,1.0,25,306.040816,18,875,49,10,917,142.0,313.263595,...,7,-32.166667,28.0,28.979399,839.81,7,10,1,10,1
4,0.5,29,30.827586,18,-11,58,-15,94,29.0,24.845827,...,29,-6.000000,10.5,22.013632,484.60,3,0,1,29,0


#### Кто предложил ничью?

In [12]:
def who_offer_draw(res,num):
    if res==0.5:
        if num % 2==0:
            return 0 # код для черных
        else:
            return 1 # код для белых
    else:
        return -1 # никто не предлагал
        
features['OfferDraw']=features[['Result','FullMoves']].apply(lambda x: who_offer_draw(*x),axis=1)
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,MeanBlackEndDeltaScore,MedianBlackEndDeltaScore,StdBlackEndDeltaScore,VarianceBlackEndDeltaScore,ModeBlackEndDeltaScore,FirstAdvantageInd,IsFirstAdvantageRealised,LastAdvantageInd,IsLastAdvantageRealised,OfferDraw
0,0.5,19,23.947368,3,54,38,-26,73,20.5,26.431587,...,-1.000000,-1.0,-1.000000,-1.00,-1,0,1,19,0,0
1,0.0,53,-692.198113,18,-11544,106,-11544,93,63.0,2286.131735,...,-428.235294,19.5,1643.924055,2702486.30,5,29,1,29,1,-1
2,1.0,39,49.246753,31,196,77,-30,198,37.0,51.525093,...,-1.450000,8.0,14.623526,213.85,8,33,1,33,1,-1
3,1.0,25,306.040816,18,875,49,10,917,142.0,313.263595,...,-32.166667,28.0,28.979399,839.81,7,10,1,10,1,-1
4,0.5,29,30.827586,18,-11,58,-15,94,29.0,24.845827,...,-6.000000,10.5,22.013632,484.60,3,0,1,29,0,0


# Это не доделано
#### Совершались ли "грубые" ошибки? (Изменение в оценке позиции > 100) 
+ход, на котором была совершена грубая ошибка

+поиск количества ошибок (Количество раз, когда оценка менялась более, чем на 50)

In [13]:
for i in range(len(data.Scores[1])):
    print("i={} :{}".format(i,data.Scores[1][i]))

i=0 :26
i=1 :51
i=2 :68
i=3 :57
i=4 :65
i=5 :77
i=6 :48
i=7 :93
i=8 :61
i=9 :63
i=10 :63
i=11 :58
i=12 :53
i=13 :46
i=14 :69
i=15 :29
i=16 :30
i=17 :27
i=18 :-2
i=19 :12
i=20 :-11
i=21 :0
i=22 :-17
i=23 :24
i=24 :5
i=25 :15
i=26 :16
i=27 :31
i=28 :44
i=29 :25
i=30 :18
i=31 :20
i=32 :27
i=33 :26
i=34 :18
i=35 :14
i=36 :18
i=37 :20
i=38 :10
i=39 :2
i=40 :7
i=41 :22
i=42 :-28
i=43 :-26
i=44 :-30
i=45 :-34
i=46 :-29
i=47 :-28
i=48 :-49
i=49 :-55
i=50 :-51
i=51 :-50
i=52 :-82
i=53 :-74
i=54 :-83
i=55 :-77
i=56 :-92
i=57 :-72
i=58 :-105
i=59 :-99
i=60 :-106
i=61 :-93
i=62 :-129
i=63 :-79
i=64 :-87
i=65 :-63
i=66 :-66
i=67 :-23
i=68 :-34
i=69 :-47
i=70 :-37
i=71 :-49
i=72 :-76
i=73 :-110
i=74 :-90
i=75 :-116
i=76 :-193
i=77 :-103
i=78 :-144
i=79 :-106
i=80 :-108
i=81 :-69
i=82 :-80
i=83 :-93
i=84 :-96
i=85 :-86
i=86 :-105
i=87 :-111
i=88 :-138
i=89 :25
i=90 :-342
i=91 :-335
i=92 :-507
i=93 :-534
i=94 :-523
i=95 :-494
i=96 :-2427
i=97 :-3977
i=98 :-5009
i=99 :-5103
i=100 :-6282
i=101 :-1273
i=

In [16]:
diffs=np.diff(data.Scores[1])

In [17]:
def find_erros(x):
    count_erros=0
    count_blunders=0
    for i in range(len(x)-1):
        if abs(x[i+1]-x[i]) >=100:
            count_blunders+=1
            is_blunders=1
            ind_first_blunders=i
        if abs(x[i+1]-x[i]) >=50:
            count_erros+=1
            is_erros=1
            ind_first_erros=i
    return diffs

diffs=delta(data.Scores[1])

i=1 - 0 :25
i=2 - 1 :17
i=3 - 2 :-11
i=4 - 3 :8
i=5 - 4 :12
i=6 - 5 :-29
i=7 - 6 :45
i=8 - 7 :-32
i=9 - 8 :2
i=10 - 9 :0
i=11 - 10 :-5
i=12 - 11 :-5
i=13 - 12 :-7
i=14 - 13 :23
i=15 - 14 :-40
i=16 - 15 :1
i=17 - 16 :-3
i=18 - 17 :-29
i=19 - 18 :14
i=20 - 19 :-23
i=21 - 20 :11
i=22 - 21 :-17
i=23 - 22 :41
i=24 - 23 :-19
i=25 - 24 :10
i=26 - 25 :1
i=27 - 26 :15
i=28 - 27 :13
i=29 - 28 :-19
i=30 - 29 :-7
i=31 - 30 :2
i=32 - 31 :7
i=33 - 32 :-1
i=34 - 33 :-8
i=35 - 34 :-4
i=36 - 35 :4
i=37 - 36 :2
i=38 - 37 :-10
i=39 - 38 :-8
i=40 - 39 :5
i=41 - 40 :15
i=42 - 41 :-50
i=43 - 42 :2
i=44 - 43 :-4
i=45 - 44 :-4
i=46 - 45 :5
i=47 - 46 :1
i=48 - 47 :-21
i=49 - 48 :-6
i=50 - 49 :4
i=51 - 50 :1
i=52 - 51 :-32
i=53 - 52 :8
i=54 - 53 :-9
i=55 - 54 :6
i=56 - 55 :-15
i=57 - 56 :20
i=58 - 57 :-33
i=59 - 58 :6
i=60 - 59 :-7
i=61 - 60 :13
i=62 - 61 :-36
i=63 - 62 :50
i=64 - 63 :-8
i=65 - 64 :24
i=66 - 65 :-3
i=67 - 66 :43
i=68 - 67 :-11
i=69 - 68 :-13
i=70 - 69 :10
i=71 - 70 :-12
i=72 - 71 :-27
i=73 - 72

# Сделать подобие ECO
"Я просто определил первые три хода как категориальную переменную, если в этом открытии было как минимум 50 игр. Для тех открытий, в которых была только одна запись, я просто назвал это открытие " редкий », 2-10 был« необычным », а 11-50 был« обычным ». (Можно использовать популярность кода ECO и вычислять среднее число игроков по данному коду.)"

# Признаки, извлеченные из игр
### Добавить еще признаков (мб по цветам)
###### (и назвать нормально)
+добавить подсчет материала
(Количество ходов с материальным преимуществом/недостатком)
(Размер преимущества/недостатка в конце)

In [13]:
from chess import QUEEN,PAWN,KNIGHT,BISHOP,ROOK,KING, WHITE,BLACK

def extract_game_features(uci):
    game = pgn.Game()
    node = game.add_variation(chess.Move.from_uci(uci[0]))
    for move in uci[1:]:
        node = node.add_variation(chess.Move.from_uci(move))

    first_check = True
    first_queen_move = True

    game_features = defaultdict(int)
    
    node = game
    while node.variations:
        move = node.variation(0).move

        board = node.board()

        # какая фигура пошла на этом ходу и что съела
        moved_piece = board.piece_type_at(move.from_square)
        captured_piece = board.piece_type_at(move.to_square)
        
        # если это был ферзь (на каком ходу)
        if moved_piece == QUEEN and first_queen_move:
            game_features['queen_moved_at'] = board.fullmove_number
            first_queen_move = False
            
        # если съели ферзя (на каком ходу)
        if captured_piece == QUEEN:
            game_features['queen_changed_at'] = board.fullmove_number

        if move.promotion: # превращение пешки (сколько пешек превратилось)
            game_features['promotion'] += 1
        if board.is_check(): # количество шахов в партии
            game_features['total_checks'] += 1
            if first_check:
                game_features['first_check_at'] = board.fullmove_number
                first_check = False
                
        # рокировка (на каком ходу)
        if board.is_kingside_castling(move) and board.turn== WHITE:
            game_features['white_king_castle'] = board.fullmove_number
        elif board.is_queenside_castling(move) and board.turn== WHITE:
            game_features['white_queen_castle'] = board.fullmove_number
        elif board.is_kingside_castling(move) and board.turn== BLACK:
            game_features['black_king_castle'] = board.fullmove_number
        elif board.is_queenside_castling(move) and board.turn== BLACK:
            game_features['black_queen_castle'] = board.fullmove_number
            
        # количество взятий на проходе
        if board.is_en_passant(move): 
            game_features['en_passant'] += 1
            
        node = node.variation(0)
        
    #board = node.board()
    # анализ итоговой позиции 
    if board.is_checkmate(): #шах
        game_features['is_checkmate'] += 1
    if board.is_stalemate(): # мат 
        game_features['is_stalemate'] += 1
    # проверяет, есть ли у одной из сторон достаточно материала для победы (false, если, например, один конь остался)
    if board.is_insufficient_material():
        game_features['insufficient_material'] += 1
    # Проверка на правило 50-ти ходов или троекратное повторение
    if board.can_claim_draw():
        game_features['can_claim_draw'] += 1

    # фигуры в итоговой позиции
    piece_placement = board.fen().split()[0]
    end_pieces = Counter(x for x in piece_placement if x.isalpha())

    # количество фигур в итоговой позиции
    game_features.update({'end_' + piece: cnt
                     for piece, cnt in end_pieces.items()})
    
    return game_features

all_features = []
for uci in data.UCI:
    ext_features = extract_game_features(uci)
    all_features.append(ext_features)

vec = DictVectorizer()
game_feat = vec.fit_transform(all_features)
#print(all_features)
#print(game_feat)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
print(type(all_features))
print(len(all_features))

print(data.UCI.shape)

print(all_features[0])

<class 'list'>
24255
(24255,)
defaultdict(<class 'int'>, {'black_king_castle': 5, 'white_king_castle': 7, 'queen_moved_at': 9, 'total_checks': 1, 'first_check_at': 14, 'queen_changed_at': 19, 'end_r': 2, 'end_k': 1, 'end_p': 6, 'end_b': 1, 'end_n': 1, 'end_q': 1, 'end_N': 1, 'end_P': 6, 'end_Q': 1, 'end_B': 1, 'end_R': 2, 'end_K': 1})


# Подумать над тем, чтобы добавить веса наиболее важным признакам
(посчитать корреляции)

+(Длинные игры вероятно более важные для понимания мастерства игры, поэтому умножение таких функций, как дисперсия и среднее, на количество ходов может быть полезно)

In [33]:
features.to_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv', sep=',',index=False)